In [21]:
# Import

from ngsolve import *
from ngsolve.meshes import MakeStructured2DMesh
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.geom2d import SplineGeometry

In [22]:
# Nitsche Hodge Laplace Method for 1-forms

def hodgeLaplace1Forms(mesh,
                       f = CF((0,0)), # this is the right hand side f
                       order = 1,
                       C_w = 1,
                       dirichletBnd = None,
                       gOnDirichletBnd = None,
                       ): 

    hcurl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl) space
    h1 = H1(mesh, order=order)     # For 0-forms, H1 space
    fes = h1 * hcurl
    (p, u), (q, v) = fes.TnT()

    n = specialcf.normal(mesh.dim)
    t = specialcf.tangential(mesh.dim)
    h = specialcf.mesh_size
    
    gamma_n_u = -curl(u)*t
    gamma_n_v = -curl(v)*t

    gamma_p_v = v - n*(v*n)
    gamma_p_u = u - n*(u*n)

    B, F  = BilinearForm(fes), LinearForm(fes)
    
    B += curl(u) * curl(v) * dx
    B += grad(p) * v * dx
    B += u * grad(q) * dx
    B += - p * q * dx

    F += f * v * dx

    for boundary_name, boundary_g in zip(dirichletBnd, gOnDirichletBnd):
        dS_dirichlet = ds(skeleton=True, definedon=mesh.Boundaries(boundary_name))
        gamma_p_g = boundary_g - n*(boundary_g*n)

        F += (C_w/h) * gamma_p_g * gamma_p_v * dS_dirichlet
        F += gamma_n_v * gamma_p_g * dS_dirichlet
        F += (boundary_g*n) * q * ds(definedon=mesh.Boundaries(boundary_name))
        
        B += (C_w/h) * gamma_p_u * gamma_p_v * dS_dirichlet
        B += gamma_n_v * gamma_p_u * dS_dirichlet
        B += gamma_p_v * gamma_n_u * dS_dirichlet

    with TaskManager():
        B.Assemble()
        F.Assemble()
        sol = GridFunction(fes)
        res = F.vec-B.mat * sol.vec
        inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
        sol.vec.data += inv * res
        gf_p , gf_u = sol.components
        
    return gf_u, gf_p

In [23]:
# Create geometry

def createReEntrantCornerGeo(h, x_offset, y_offset):
    wp = WorkPlane()
    wp.MoveTo(0+x_offset, 0+y_offset)
    wp.LineTo(0+x_offset, -0.5+y_offset, name="reEntrantVertical")
    wp.LineTo(0.5+x_offset, -0.5+y_offset, name="bottom")
    wp.LineTo(0.5+x_offset, 0.5+y_offset, name="right")
    wp.LineTo(-0.5+x_offset, 0.5+y_offset, name="top")
    wp.LineTo(-0.5+x_offset, 0+y_offset, name="left")
    wp.LineTo(0+x_offset, 0+y_offset, name="reEntrantHorizontal")
    wp.Close()
    shape = wp.Face()

    geometry = OCCGeometry(shape, dim=2)
    reEntrantCornerGeo = Mesh(geometry.GenerateMesh(maxh=h))
    return reEntrantCornerGeo

In [24]:
mesh = createReEntrantCornerGeo(0.05, 0, 0)
t, n = specialcf.tangential(mesh.dim), specialcf.normal(mesh.dim)

C_w = 100
order = 2

f_HL = 1 * CF((3, -3))
bndList_HL = ["left", "bottom", "top", "right", "reEntrantVertical", "reEntrantHorizontal"]
gList_HL = [-0.1*n, 0.1*n, CF((0,0)), CF((0,0)), CF((0,0)), CF((0,0))]

gfu, gfp = hodgeLaplace1Forms(mesh, f_HL, order, C_w, bndList_HL, gList_HL)

In [25]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…

BaseWebGuiScene